In [1]:
import tensorflow as tf

import keras.backend as K
import keras
K.set_image_data_format('channels_first')
from keras.backend.tensorflow_backend import set_session

from keras import callbacks
from keras.losses import categorical_crossentropy
from keras.optimizers import *

from keras.callbacks import LearningRateScheduler

from keras.layers import *
from keras.models import Model, Sequential

import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  
config.gpu_options.per_process_gpu_memory_fraction = 0.95
set_session(tf.Session(config=config))


trainDataPath = "./LaneDetection/seg_data/train_data_new_noRollAxis.npy"
trainLabelPath = "./LaneDetection/seg_data/train_label_new_noRollAxis.npy"
valDataPath = "./LaneDetection/seg_data/val_data_new_noRollAxis.npy"
valLabelPath = "./LaneDetection/seg_data/val_label_new_noRollAxis.npy"
path2write = "./LaneDetection/seg_data/unet_depthwise/"
filepath=path2write + 'unet-norm-{epoch:02d}-trainLoss-{loss:02f}-trainAcc-{acc:.2f}-trainFgDice-{fgDiceScore:.2f}-valLoss-{val_loss:02f}-valAcc{val_acc:.2f}-valFgDice{val_fgDiceScore:.2f}.h5'

classes = 2
depth = 3
height = 720
width = 1280
batch_size = 1
save_after_epochs = 1

C:\Users\ameya.deshpande\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ameya.deshpande\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ameya.deshpande\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\U

In [2]:
def relu6(x):
    return K.relu(x, max_value=6)

def _depthwise_conv_block(inputs, pointwise_conv_filters, alpha = 1.0,
                          depth_multiplier=1, strides=(1, 1), block_id=1):

    pointwise_conv_filters = int(pointwise_conv_filters * alpha)

    x = ZeroPadding2D((1, 1), data_format='channels_first')(inputs)
    x = DepthwiseConv2D((3, 3), data_format='channels_first',
                        padding='valid',
                        depth_multiplier=depth_multiplier,
                        strides=strides,
                        use_bias=False)(x)
    x = BatchNormalization(
        axis=1)(x)
    x = Activation(relu6)(x)

    x = Conv2D(pointwise_conv_filters, (1, 1), data_format='channels_first',
               padding='same',
               use_bias=False,
               strides=(1, 1))(x)
    x = BatchNormalization(axis=1)(x)
    return Activation(relu6)(x)



def unet_depthwiseConv(pretrained_weights = None,input_size = (3, height, width)):
    
    
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = _depthwise_conv_block(pool1, pointwise_conv_filters = 64)
    conv2 = _depthwise_conv_block(conv2, pointwise_conv_filters = 64)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = _depthwise_conv_block(pool2, pointwise_conv_filters = 128)
    conv3 = _depthwise_conv_block(conv3, pointwise_conv_filters = 128)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = _depthwise_conv_block(pool3, pointwise_conv_filters = 256)
    conv4 = _depthwise_conv_block(conv4, pointwise_conv_filters = 256)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = _depthwise_conv_block(pool4, pointwise_conv_filters = 512)
    conv5 = _depthwise_conv_block(conv5, pointwise_conv_filters = 512)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 1)
    conv6 = _depthwise_conv_block(merge6, pointwise_conv_filters = 256)
    conv6 = _depthwise_conv_block(conv6, pointwise_conv_filters = 256)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 1)
    conv7 = _depthwise_conv_block(merge7, pointwise_conv_filters = 128)
    conv7 = _depthwise_conv_block(conv7, pointwise_conv_filters = 128)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 1)
    conv8 = _depthwise_conv_block(merge8, pointwise_conv_filters = 64)
    conv8 = _depthwise_conv_block(conv8, pointwise_conv_filters = 64)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 1)
    conv9 = _depthwise_conv_block(merge9, pointwise_conv_filters = 32)
    conv9 = _depthwise_conv_block(conv9, pointwise_conv_filters = 32)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    
    reshape = Reshape((2, width*height), input_shape = (2, height, width))(conv9)

    permut = Permute((2,1))(reshape)
    softmax = Activation('softmax')(permut)
    model = Model(inputs= [inputs], outputs= [softmax])
    

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [4]:
#function to generate categorical label from single channel label
def binarylab(labels):
    
    #Define an Empty Array 
    x = np.zeros([height, width, classes],dtype="uint8")
    
    #Read Each pixel label and put it into corresponding label plane
    for i in range(height):
        for j in range(width):
            x[i,j,labels[i][j]] = 1
    return x

In [5]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    """
    lr = 1e-3
    if epoch > 30:
        lr *= 1e-1
    if epoch > 60:
        lr *= 1e-1
    if epoch > 100:
        lr *= 1e-1
    if epoch > 200:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [6]:
def sampleMeanStd(img):
    
    img  = img.astype("float32")
    
    b_ch=np.mean(img[:,:,0])
    g_ch=np.mean(img[:,:,1])
    r_ch=np.mean(img[:,:,2])

    #Individual channel-wise mean subtraction
    img -= np.array((b_ch,g_ch,r_ch))
    
    b_ch=np.std(img[:,:,0])
    g_ch=np.std(img[:,:,1])
    r_ch=np.std(img[:,:,2])
    
    img /= np.array((b_ch,g_ch,r_ch))
    
    return img


#Data generator class
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, label, batch_size = batch_size, dim = (height, width), depth = depth,
                 num_classes = classes, shuffle = True):
        
        self.data = data
        self.label = label
        self.batch_size = batch_size
        self.dim = dim
        self.depth = depth
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.indexes = np.arange(np.shape(self.data)[0])


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(np.shape(self.data)[0] / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        #print('Index : ' + str(index))
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        
        X_batch = np.empty((self.batch_size, self.depth, self.dim[0], self.dim[1]))
        y_batch = np.empty((self.batch_size, self.dim[0]*self.dim[1], self.num_classes), dtype = np.uint8)
        
        for i, index in enumerate(indexes):
            
            #bgr image array
            x_temp = self.data[index, :, :, :]
            y_temp = self.label[index, :, :]
                 
            #perform simple normalisation
            x_temp = sampleMeanStd(x_temp)
                
            #perform categorical
            y_temp = binarylab(y_temp)          
            y_temp = np.reshape(y_temp, (self.dim[0]*self.dim[1], self.num_classes))
                
            X_batch[i, ] = np.rollaxis((x_temp),2)
            y_batch[i, ] = y_temp
        
        return X_batch, y_batch
        
    def on_epoch_end(self):
        
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [7]:
#Dice score, to be found out after each epoch
def DiceScore(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice =  (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return dice

def fgDiceScore(y_true, y_pred):
    y_true_f = K.flatten(y_true[:, :, 1])
    y_pred_f = K.flatten(y_pred[:, :, 1])
    intersection = K.sum(y_true_f * y_pred_f)
    dice =  (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return dice

def DiceLoss(targets, inputs, smooth=1e-6): 
    #flatten label and prediction tensors 
    inputs = K.flatten(inputs)
    targets = K.flatten(targets) 
    intersection = K.sum(K.dot(targets, inputs)) 
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth) 
    return 1 - dice


In [8]:
#Actual training part

#make a directory to save logs
if not os.path.exists(path2write):
    os.makedirs(path2write)

            
#Load numpy arrays from the directory
trainData = np.load(trainDataPath)
trainLabel = np.load(trainLabelPath)
valData =  np.load(valDataPath)
valLabel = np.load(valLabelPath)

print('Training samples :', np.shape(trainData))
print('Validation samples :', np.shape(valData))

#make train and val data generators
train_data_generator = DataGenerator(data = trainData, label = trainLabel)
val_data_generator = DataGenerator(data = valData, label = valLabel)

#Load model
model = unet_depthwiseConv(pretrained_weights = "./LaneDetection/unet_model_depth1/unet_model_depth1-norm-137-trainLoss-0.008836-trainAcc-1.00-trainFgDice-0.84-valLoss-0.181117-valAcc0.98-valFgDice0.26.h5",
                           input_size = (3, height, width))

print(model.summary())  
print ("Compiling Model...")   

modelCheck = callbacks.ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1, 
                                        save_best_only = False, mode = 'auto', 
                                        period = save_after_epochs)
#optimizer
#opt = RMSprop()
opt = Adam(lr=lr_schedule(0))

tensorboardCallback = callbacks.TensorBoard(log_dir = path2write, histogram_freq = 0, 
                                            write_graph = True, write_images = True)

#compiling the model
model.compile(loss = categorical_crossentropy, optimizer=opt, metrics=["accuracy", DiceScore, fgDiceScore])
lr_scheduler = LearningRateScheduler(lr_schedule)

print ("Training the Model...")

unn2 = model.fit_generator(generator = train_data_generator, epochs = 300, verbose = 1, 
                               callbacks = [modelCheck, tensorboardCallback, lr_scheduler], 
                               validation_data = val_data_generator, 
                               use_multiprocessing = False, initial_epoch = 0)
    
print ("Dumping Weights to file...")

# After training for given number of epochs save the model weights on the disk
#model.save_weights(path2save, overwrite=True)
print ("Models Saved :-)")    

Training samples : (300, 720, 1280, 3)
Validation samples : (100, 720, 1280, 3)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 720, 1280) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 720, 1280 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 720, 1280 9248        conv2d_1[0][0]                   
_______________________

KeyboardInterrupt: 